In [4]:
!pip install git+https://github.com/LUFFY-02/cuda.git

  Cloning https://github.com/LUFFY-02/cuda.git to /tmp/pip-req-build-j682gwa_
  Running command git clone --filter=blob:none --quiet https://github.com/LUFFY-02/cuda.git /tmp/pip-req-build-j682gwa_
  Resolved https://github.com/LUFFY-02/cuda.git to commit aa845e22f008c8f70745bd6d67f6c42c50ac5363
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4289 sha256=c8c0d7f3973e09701787dcd592a00d47cc5a9530ec42de716e74e18d8d69393d
  Stored in directory: /tmp/pip-ephem-wheel-cache-t5itwoks/wheels/2f/c4/fc/9c6b8e2f4a3330ee0b57405367cbbd5a1245b4bf2ca2b5b5b9
Successfully built NVCCPlugin


In [5]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [6]:
%%cu
// checking that cuda is working or not
#include <iostream>
int main(){
  std::cout << "Hello World\n";
  return 0;
}

Hello World



In [18]:
%%cu
#include <iostream>
#include<chrono>
using namespace std;


// CUDA code to multiply matrices
__global__ void multiply(int* A, int* B, int* C, int size) {
    // Uses thread indices and block indices to compute each element
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < size && col < size) {
        int sum = 0;
        for (int i = 0; i < size; i++) {
            sum += A[row * size + i] * B[i * size + col];
        }
        C[row * size + col] = sum;
    }
}


void initialize(int* matrix, int size) {
    for (int i = 0; i < size * size; i++) {
        matrix[i] = rand() % 10;
    }
}


void print(int* matrix, int size) {
    for (int row = 0; row < size; row++) {
        for (int col = 0; col < size; col++) {
            cout << matrix[row * size + col] << " ";
        }
        cout << '\n';
    }
    cout << '\n';
}

int main() {
    int* A, * B, * C;

    int N = 4;

    int matrixSize = N * N;
    size_t matrixBytes = matrixSize * sizeof(int);

    A = new int[matrixSize];
    B = new int[matrixSize];
    C = new int[matrixSize];

    initialize(A, N);
    initialize(B, N);
    cout << "Matrix A: \n";
    print(A, N);

    cout << "Matrix B: \n";
    print(B, N);


    int* X, * Y, * Z;
    // Allocate space
    cudaMalloc(&X, matrixBytes);
    cudaMalloc(&Y, matrixBytes);
    cudaMalloc(&Z, matrixBytes);

    // Copy values from A to X
    cudaMemcpy(X, A, matrixBytes, cudaMemcpyHostToDevice);

    // Copy values from A to X and B to Y
    cudaMemcpy(Y, B, matrixBytes, cudaMemcpyHostToDevice);

    // Threads per CTA dimension
    int THREADS = 2;

    // Blocks per grid dimension (assumes THREADS divides N evenly)
    int BLOCKS = N / THREADS;

    // Use dim3 structs for block  and grid dimensions
    dim3 threads(THREADS, THREADS);
    dim3 blocks(BLOCKS, BLOCKS);

    // Launch kernel
    auto start_sequential = std::chrono::steady_clock::now();
    multiply<<<blocks, threads>>>(X, Y, Z, N);
    auto end_sequential = std::chrono::steady_clock::now();
    std::chrono::duration<double> elapsed_seconds_sequential = end_sequential - start_sequential;
    //end=omp_get_wtime();
    cudaMemcpy(C, Z, matrixBytes, cudaMemcpyDeviceToHost);

    std::cout << "Execution time: " << elapsed_seconds_sequential.count() << "s\n";


    //cout<<end-start<<endl;

    cout << "Multiplication of matrix A and B: \n";
    print(C, N);

    delete[] A;
    delete[] B;
    delete[] C;

    cudaFree(X);
    cudaFree(Y);
    cudaFree(Z);

    return 0;
}


Matrix A: 
3 6 7 5 
3 5 6 2 
9 1 2 7 
0 9 3 6 

Matrix B: 
0 6 2 6 
1 8 7 9 
2 0 2 3 
7 5 9 2 

Execution time: 0.000232867s
Multiplication of matrix A and B: 
55 91 107 103 
31 68 71 85 
54 97 92 83 
57 102 123 102 


